In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', None)

import re

import seaborn as sea
import matplotlib.pyplot as plt

from nanoporePipelineCommon import *

print("imports done")

In [ ]:
reads_df_genes_raw, compressed_df_genes_raw = load_and_merge_lib_parquets(["xrn-1-5tera", "xrn-1-5tera-smg-6"], drop_sub_n=1, add_tail_groupings=False, drop_failed_polya=False, group_by_t5=True)
print("library load done.")

In [ ]:
reads_df = reads_df_genes_raw.copy()
compressed_df = compressed_df_genes_raw.copy()

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

def _make_rectangle_patch(genome_start, length, y_center, thickness, color='gray'):
    return Rectangle((genome_start, y_center-(thickness/2)), length, thickness,
                     facecolor=color,
                     edgecolor=color,
                     fill=True,
                     lw=0)

def _add_patches_from_cigars_and_gen_pos(axes, cigar, gen_start, y, color='black', tail_length=None):
    # Parse the cigar string
    parsed_cigar = re.findall(rf'(\d+)([MDNSIX])', cigar)
    mdn_nums = [int(num) for num, char in parsed_cigar if char in "MDN"]
    read_end = gen_start + sum(mdn_nums)
    
    genome_loc = gen_start
    
    rectangle_patch_list = []
    first_n_length = 0
    for length, code in parsed_cigar:
        length = int(length)
        if code == 'S':
            if len(rectangle_patch_list) == 0:
                # genome_loc += length  # <- this was dumb and wrong...
                pass
        elif code == 'M':
            rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.8))
            genome_loc += length
        elif code == 'D':
            if length < 50:
                rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.8))
            else:
                rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.001))
            genome_loc += length
        elif code == 'I':
            pass
        elif code == 'N':
            if first_n_length == 0:
                first_n_length = length
                # print(f"First N of length {length}, starting at {genome_loc}")
            rectangle_patch_list.append(_make_rectangle_patch(genome_loc, length, y, thickness=0.001))
            genome_loc += length
    if read_end - gen_start < 1500:
        axes.add_collection(PatchCollection(rectangle_patch_list, color=color))
        if isinstance(tail_length, float):
            axes.add_patch(_make_rectangle_patch(genome_loc, tail_length, y, thickness=0.4, color='green'))
            genome_loc += tail_length
        return read_end - gen_start, first_n_length
    else:
        return None

def _row_apply_plot_cigar(row, axes):
    index = row.name
    cigar = row.cigar
    gen_start = row.chr_pos
    is_adapted = row.t5
    polya_length = row.polya_length
    
    if is_adapted == '-':
        color='black'
    else:
        color='red'
    return _add_patches_from_cigars_and_gen_pos(axes, cigar, gen_start, index, color=color, tail_length=polya_length)

In [ ]:
def plot_reads(gene_of_interest, reads_df, save_dir=None, save_suffix="", t5_pos_count=10, t5_neg_count=25, plot_width_and_height=(25,5), subsample_fraction=0.05):
    subsampled_reads_df = reads_df.sample(n=subsample_fraction)
    all_gene_reads = subsampled_reads_df.query(f"gene_name == '{gene_of_interest}'")
    gene_df_t5_pos = all_gene_reads.query("t5 == '+'")  #.sample(t5_pos_count)
    gene_df_t5_neg = all_gene_reads.query("t5 == '-'")  #.sample(t5_neg_count)
    gene_df = pd.concat([gene_df_t5_pos, gene_df_t5_neg])
    
    gene_df = gene_df.sort_values(["t5", "chr_pos", "read_length"], ascending=False).reset_index(drop=True)
    
    plt.style.use('default')
    fig, ax = plt.subplots()
    tqdm.pandas(desc="First pass to extract the length of the first intron called")
    gene_df[['genomic_read_length', 'first_n_length']] = gene_df.progress_apply(lambda row: _row_apply_plot_cigar(row, ax), axis=1, result_type='expand')
    fig, ax = None, None
    
    plt.style.use('default')
    fig, ax = plt.subplots(figsize=plot_width_and_height)
    
    tqdm.pandas(desc="Second pass to actually plot everything")
    gene_df = gene_df.sort_values(["t5", "first_n_length", "chr_pos"], ascending=[False, False, False]).reset_index(drop=True)
    gene_df.progress_apply(lambda row: _row_apply_plot_cigar(row, ax), axis=1, result_type='expand')

    
    
    
    number_of_plotted_reads = gene_df.shape[0]
    ax.set_ylim(-1, number_of_plotted_reads)
    
    if number_of_plotted_reads < 50:
        num_long_reads_to_check = 1
    else:
        num_long_reads_to_check = int(number_of_plotted_reads/10)
    
    min_genomic_start = gene_df.chr_pos.min()
    max_genomic_length = gene_df.nlargest(num_long_reads_to_check, 'genomic_read_length').genomic_read_length.min()
    
    ax.set_xlim(min_genomic_start-25, min_genomic_start+max_genomic_length+100)
    # ax.set_xlim(min_genomic_start, min_genomic_start+300)
    
    plt.xticks([])
    plt.yticks([])
    if isinstance(save_dir, str):
        save_path = f"{save_dir}/{get_dt(for_file=True)}_{gene_of_interest}{save_suffix}"
        print(f"Saving plot to {save_path} + .png/.svg...")
        plt.savefig(save_path + ".svg")
        plt.savefig(save_path + ".png")
    # plt.show()
    return gene_df

In [ ]:
fraction = 400000
for i in range(1):
    df1 = plot_reads("ubl-1", reads_df.query("lib == 'xrn-1-5tera'"), save_dir="./outputDir", save_suffix=f"_wt_{i+1}", subsample_fraction=fraction)
    # df2 = plot_reads("ubl-1", reads_df.query("lib != 'xrn-1-5tera'"), save_dir="./outputDir", save_suffix=f"_smg-6_{i+1}", t5_pos_count=2, subsample_fraction=fraction)